In [2]:
from os.path import abspath
from pyspark.sql import SparkSession

# set default location for [spark-warehouse]
warehouse_location = abspath('spark-warehouse')


In [3]:
from pyspark.sql.avro.functions import from_avro, to_avro
import pyspark.sql.functions as f
from urllib.request import urlopen
import json
from pyspark.sql.types import StringType, StructType
from dateutil import parser
from delta.tables import *
from datetime import datetime

In [4]:
spark = (
    SparkSession
    .builder
    .appName("etl-test")
    .config("spark.jars.packages", "org.apache.spark:spark-avro_2.12:3.1.1,io.delta:delta-core_2.12:1.0.0")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .config("spark.sql.debug.maxToStringFields","1000")
    .config("spark.kryoserializer.buffer.max","128m")
    .config("spark.driver.memory", "24G")
    .enableHiveSupport()
    .getOrCreate()
)
spark.sparkContext.setLogLevel("WARN")

:: loading settings :: url = jar:file:/usr/local/spark-3.1.2-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
org.apache.spark#spark-avro_2.12 added as a dependency
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-7a567169-767c-496f-aaa1-1aac7c46476c;1.0
	confs: [default]
	found org.apache.spark#spark-avro_2.12;3.1.1 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found io.delta#delta-core_2.12;1.0.0 in central
	found org.antlr#antlr4;4.7 in central
	found org.antlr#antlr4-runtime;4.7 in central
	found org.antlr#antlr-runtime;3.5.2 in central
	found org.antlr#ST4;4.0.8 in central
	found org.abego.treelayout#org.abego.treelayout.core;1.0.3 in central
	found org.glassfish#javax.json;1.0.4 in central
	found com.ibm.icu#icu4j;58.2 in central
:: resolution report :: resolve 556ms :: artifacts dl 12ms
	:: modules in use:
	com.ibm.icu#icu4j;58.2 from central in [default]
	io.delta#delta-core_2.12;1.0.0 from c

In [5]:
conf = spark.sparkContext._jsc.hadoopConfiguration()
conf.set("fs.gs.project.id", "hering-datalake-prod")
conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
conf.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")

In [6]:
def genereteSymlinkManifest(spark:SparkSession, bucket:str) -> None:
    """
    Gera o arquivo symlink manifest para a tabela Delta do bucket de referencia
    """
    delta_table = DeltaTable.forPath(spark, bucket)
    delta_table.generate("symlink_format_manifest")

def pathExists(path, spark:SparkSession):
    sc = spark.sparkContext
    fs = sc._jvm.org.apache.hadoop.fs.FileSystem.get(
        sc._jvm.java.net.URI.create("gs://" + path.split("/")[2]),
        sc._jsc.hadoopConfiguration(),
    )
    return fs.exists(sc._jvm.org.apache.hadoop.fs.Path(path))

def lastDate(delta_path:str, spark:SparkSession):
    if not pathExists(delta_path + 'job_control/last_date', spark):
        try:
            df_temp = (
                spark
                .read
                .format("delta")
                .load(delta_path)
            )

            list_result = (
                df_temp
                .agg({"timestamp_kafka": "max"}).collect()
            )

            last_date = str(list_result[0].asDict()['max(timestamp_kafka)'])
        except:
            last_date = ('01-01-2021 12:00AM')

        df = spark.read.json(spark.sparkContext.parallelize([{'last_date': last_date}]))
        df.write.mode('overwrite').csv(delta_path + 'job_control/last_date', header = 'true')
    else:
        df = spark.read.format("csv").load(delta_path + 'job_control/last_date', header = 'true')
        last_date = df.collect()[0].asDict()['last_date']

    logger.info('Last date: ' + last_date)

    return parser.parse(last_date)

def updateLastDate(delta_path:str, df:DataFrame, spark:SparkSession):
    list_result = (
        df
        .agg({"timestamp_kafka": "max"}).collect()
    )

    last_date = str(list_result[0].asDict()['max(timestamp_kafka)'])
    df = spark.read.json(spark.sparkContext.parallelize([{'last_date': last_date}]))
    df.write.mode('overwrite').csv(delta_path + 'job_control/last_date', header = 'true')

    return last_date


In [7]:
def getOperationsCounts(df:DataFrame, spark:SparkSession):
    op_count_list =(
        df
        .groupBy('__op')
        .count()
        .collect()
    )
    
    new_data_count = 0
    updating_data_count = 0
    deleting_data_count = 0

    if len(op_count_list) > 0:
        for i in op_count_list:
            dic_temp = i.asDict()
            if (dic_temp['__op'] == 'c' or dic_temp['__op'] == 'r'):
                new_data_count = new_data_count + dic_temp['count']
            elif (dic_temp['__op'] == 'd'):
                deleting_data_count = deleting_data_count + dic_temp['count']
            elif (dic_temp['__op'] != 'c' and dic_temp['__op'] != 'r'):
                updating_data_count = updating_data_count + dic_temp['count']
            
    new_total_data_count = new_data_count + updating_data_count + deleting_data_count
    
    return new_total_data_count, new_data_count, updating_data_count, deleting_data_count

In [8]:
def getData(spark:SparkSession, bucket:str, date_colunm_name:str, date_begin:datetime, source_format:str='delta'):
    """
    Carrega Dataframe filtrando os dados do bucket de referência conforme 
    data de início enviada no parametro

    :param str date_begin: Data de início para filtro dos dados
    """
    df = (
        spark
        .read
        .format(source_format)
        .load(bucket)
        .where(
            (f.col("year")>date_begin.year)|
              (
                (f.col("year")==date_begin.year)&
                (
                  (f.col("month")>date_begin.month)|
                  (
                    (f.col("month")==date_begin.month)&
                    (
                      (f.col("day")>date_begin.day)|
                      (
                        (f.col("day")==date_begin.day)&
                        (f.col("hour")>=date_begin.hour)&
                        (f.col(date_colunm_name)>date_begin)
                      )
                    )
                  )
                )          
              ) 
        )
    )
    return df

In [9]:
def writeNewData(spark:SparkSession, df:DataFrame, bucket:str, destination_format:str='delta'):
    df_r = df.where((f.col('__op') == 'r')|(f.col('__op')=='c'))
    (
        df_r
        .write
        .format(destination_format)
        .mode("append")
        .option("checkpointLocation", bucket + "checkpoint")
        .save(bucket)
    )
    
    return True

In [10]:
def updateNewData(spark:SparkSession, df:DataFrame, bucket:str, pk:list=['pk_single']):
    table = DeltaTable.forPath(spark, bucket)
    df_table_u = df.where((f.col('__op') != 'r')&(f.col('__op')!='c'))
    df_table_u_unique = df_table_u.orderBy('timestamp_kafka', ascending = False).dropDuplicates(pk)
    merge_string = 'destination.' + pk[0] + ' = updates.' + pk[0]
    (
        table.alias('destination')
        .merge(df_table_u_unique.alias('updates'), merge_string)
        .whenMatchedUpdateAll()
        .whenNotMatchedInsertAll()
        .execute()
    )
    
    return True

In [11]:
def deleteNewData(spark:SparkSession, bucket:str):
    table = DeltaTable.forPath(spark, bucket)
    table.delete(col('__op') == 'd')
    
    return True

In [12]:
bronze_path = 'gs://hering-datalake-prod-southamerica-east1-bucket-bronze-zone/ecommerce/bnsvher602/linx_db/dbo/loja_venda/'
silver_path = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/ecommerce/bnsvher602/linx_db/dbo/loja_venda/'
silver_path_restricted = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/ecommerce/bnsvher602/linx_db/dbo/loja_venda_restrito/'

In [13]:
try:
    df_silver = (
        spark
        .read
        .format("delta")
        .load(silver_path)
    )

    list_result = (
        df_silver
        .agg({"timestamp_kafka": "max"}).collect()
    )

    last_date = list_result[0][0]

except:
    last_date = parser.parse("01-01-2021 12:00AM")

In [14]:
last_date = parser.parse("01-01-2022 12:00AM")

In [65]:
len(df_silver.columns)

63

In [15]:
from pyspark.sql.avro.functions import from_avro, to_avro
import pyspark.sql.functions as f
from urllib.request import urlopen
import json
from pyspark.sql.types import StringType, StructType
from dateutil import parser
from delta.tables import *

In [16]:
df_table = (
    spark
    .read
    .format("delta")
    .load(bronze_path)
    .where((f.col("year")>=last_date.year)&(f.col("month")>=last_date.month)&(f.col("day")>=last_date.day))
    .where(f.col("timestamp_kafka")>last_date)
)

In [60]:
df_table.printSchema()

root
 |-- offset: long (nullable = true)
 |-- timestamp_kafka: timestamp (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- hour: integer (nullable = true)
 |-- CODIGO_FILIAL: integer (nullable = true)
 |-- TICKET: string (nullable = true)
 |-- DATA_VENDA: timestamp (nullable = true)
 |-- CODIGO_DESCONTO: string (nullable = true)
 |-- CODIGO_CLIENTE: string (nullable = true)
 |-- PERIODO: string (nullable = true)
 |-- DATA_DIGITACAO: timestamp (nullable = true)
 |-- VENDEDOR: string (nullable = true)
 |-- OPERACAO_VENDA: string (nullable = true)
 |-- CODIGO_TAB_PRECO: string (nullable = true)
 |-- COMISSAO: double (nullable = true)
 |-- DESCONTO: double (nullable = true)
 |-- QTDE_TOTAL: double (nullable = true)
 |-- VALOR_TIKET: double (nullable = true)
 |-- VALOR_PAGO: double (nullable = true)
 |-- VALOR_VENDA_BRUTA: double (nullable = true)
 |-- VALOR_TROCA: double (nullable = true)
 |-- QTDE_TROCA_TO

In [59]:
df_table.drop(df_table.__deleted)

DataFrame[offset: bigint, timestamp_kafka: timestamp, year: int, month: int, day: int, hour: int, CODIGO_FILIAL: int, TICKET: string, DATA_VENDA: timestamp, CODIGO_DESCONTO: string, CODIGO_CLIENTE: string, PERIODO: string, DATA_DIGITACAO: timestamp, VENDEDOR: string, OPERACAO_VENDA: string, CODIGO_TAB_PRECO: string, COMISSAO: double, DESCONTO: double, QTDE_TOTAL: double, VALOR_TIKET: double, VALOR_PAGO: double, VALOR_VENDA_BRUTA: double, VALOR_TROCA: double, QTDE_TROCA_TOTAL: double, TICKET_IMPRESSO: boolean, TERMINAL: string, GERENTE_LOJA: string, GERENTE_PERIODO: string, CODIGO_FILIAL_PGTO: int, TERMINAL_PGTO: string, LANCAMENTO_CAIXA: string, TIMESTAMP: binary, VALOR_CANCELADO: double, TOTAL_QTDE_CANCELADA: double, DATA_HORA_CANCELAMENTO: timestamp, PERIODO_FECHAMENTO: string, DATA_PARA_TRANSFERENCIA: timestamp, FATOR: double, SUGESTAO_COD_FORMA_PGTO: string, VALOR_IPI: double, QTDE_ITEM: int, MOTIVO_CANCELAMENTO: int, TERMINAL_VERNDA_SUSPENSA: string, TERMINAL_VENDA_SUSPENSA: strin

In [70]:
df_table = df_table.withColumn('CODIGO_FILIAL', df_table.CODIGO_FILIAL.cast('int'))\
.withColumn('DATA_VENDA', f.to_timestamp(f.col('DATA_VENDA')))\
.withColumn('DATA_DIGITACAO', f.to_timestamp(f.col('DATA_DIGITACAO')))\
.withColumn('CODIGO_FILIAL_PGTO', df_table.CODIGO_FILIAL_PGTO.cast('int'))\
.withColumn('DATA_HORA_CANCELAMENTO', f.to_timestamp(f.col('DATA_HORA_CANCELAMENTO')))\
.withColumn('DATA_PARA_TRANSFERENCIA', f.to_timestamp(f.col('DATA_PARA_TRANSFERENCIA')))\
.withColumn('DATA_HORA_INSERT_ERP', f.to_timestamp(f.col('DATA_HORA_INSERT_ERP')))

for i in df_table.columns:
    if i not in df_silver.columns:
        df_table = df_table.drop(i)


In [73]:
len(df_table_hash.columns)

64

In [74]:
for i in df_table_hash.columns:
    if i not in df_silver.columns:
        print(i)

CPF_CGC_ECF_HASH


In [20]:
new_total_data_count, new_data_count, updating_data_count, deleting_data_count = getOperationsCounts(df_table, spark)
print('New Total Data:' + str(new_total_data_count))
print('New Data:' + str(new_data_count))
print('Updating Data:' + str(updating_data_count))
print('Deleting Data:' + str(deleting_data_count))

Exception in thread "serve-DataFrame" java.net.SocketTimeoutException: Accept timed out
	at java.base/java.net.PlainSocketImpl.socketAccept(Native Method)
	at java.base/java.net.AbstractPlainSocketImpl.accept(AbstractPlainSocketImpl.java:458)
	at java.base/java.net.ServerSocket.implAccept(ServerSocket.java:565)
	at java.base/java.net.ServerSocket.accept(ServerSocket.java:533)
	at org.apache.spark.security.SocketAuthServer$$anon$1.run(SocketAuthServer.scala:64)


New Total Data:41835432
New Data:41824252
Updating Data:11174
Deleting Data:6


In [71]:
if new_total_data_count > 0:
    print('Defining restricted table')
    df_table_restricted = (
        df_table
        .select(f.col('CODIGO_FILIAL'), f.col('TICKET'), f.col('DATA_VENDA'), f.col('CODIGO_CLIENTE'), f.col('CPF_CGC_ECF'), f.col('__op'), f.col('timestamp_kafka'))
        .withColumn("CODIGO_CLIENTE_HASH", f.sha2(f.col('CODIGO_CLIENTE'), 256))    
        .withColumn("CPF_CGC_ECF_HASH", f.sha2(f.col('CPF_CGC_ECF'), 256))
        .withColumn('pk_single', f.concat(f.col('CODIGO_FILIAL'), f.col('TICKET'), f.col('DATA_VENDA')))
    )    

    print('Defining hash table')
    df_table_hash = (
        df_table
        .withColumn("CODIGO_CLIENTE", f.sha2(f.col('CODIGO_CLIENTE'), 256))
        .withColumn("CPF_CGC_ECF_HASH", f.sha2(f.col('CPF_CGC_ECF'), 256))
        .withColumn('pk_single', f.concat(f.col('CODIGO_FILIAL'), f.col('TICKET'), f.col('DATA_VENDA')))    
    )
    
    if new_data_count > 0:
        print('Writing hash table new data ')
        print('Writing ' + str(new_data_count) + ' new data')
        writeNewData(spark, df_table_hash, silver_path, 'delta')
        
        print('Writing restrited table new data')
        print('Writing ' + str(new_data_count) + ' new data')
        writeNewData(spark, df_table_restricted, silver_path_restricted, 'delta')
        
    if updating_data_count > 0:
        print('Updating hash table data')
        print('Updating ' + str(updating_data_count) + ' data')
        updateNewData(spark, df_table_hash, silver_path, ['pk_single'])
            
        print('Updanting restrited table data')
        print('Updating ' + str(updating_data_count) + ' data')
        updateNewData(spark, df_table_restricted, silver_path_restricted, ['pk_single'])
        
    if deleting_data_count > 0:
        print('Deleting hash table data')
        print('Deleting ' + str(deleting_data_count) + ' data')
        deleteNewData(spark, silver_path)
        
        print('Deleting restrited table data')
        print('Deleting ' + str(deleting_data_count) + ' data')
        deleteNewData(spark, silver_path_restricted)

Defining restricted table
Defining hash table
Writing hash table new data 
Writing 41824252 new data


AnalysisException: A schema mismatch detected when writing to the Delta table (Table ID: bd363653-0436-4179-8c56-2c00736d27be).
To enable schema migration using DataFrameWriter or DataStreamWriter, please set:
'.option("mergeSchema", "true")'.
For other operations, set the session configuration
spark.databricks.delta.schema.autoMerge.enabled to "true". See the documentation
specific to the operation for details.

Table schema:
root
-- timestamp_kafka: timestamp (nullable = true)
-- CODIGO_FILIAL: integer (nullable = true)
-- TICKET: string (nullable = true)
-- DATA_VENDA: timestamp (nullable = true)
-- CODIGO_DESCONTO: string (nullable = true)
-- CODIGO_CLIENTE: string (nullable = true)
-- PERIODO: string (nullable = true)
-- DATA_DIGITACAO: timestamp (nullable = true)
-- VENDEDOR: string (nullable = true)
-- OPERACAO_VENDA: string (nullable = true)
-- CODIGO_TAB_PRECO: string (nullable = true)
-- COMISSAO: double (nullable = true)
-- DESCONTO: double (nullable = true)
-- QTDE_TOTAL: d